In [ ]:
conda install -c conda-forge chromadb

In [ ]:
conda install pip

In [ ]:
pip install gpt4all

In [ ]:
pip install langchain

In [ ]:
pip install langchainhub

In [ ]:
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://inciweb.nwcg.gov/incidents/rss.xml")
data = loader.load()

In [ ]:
# Load web page# Split into chunks 
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)
all_splits = text_splitter.split_documents(data)


In [ ]:
# Split into chunks 
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)
all_splits = text_splitter.split_documents(data)

In [ ]:
# Embed and store
from langchain.vectorstores import Chroma
from langchain.embeddings import GPT4AllEmbeddings
# from langchain.embeddings import OllamaEmbeddings # We can also try Ollama embeddings
vectorstore = Chroma.from_documents(documents=all_splits,
                                    embedding=GPT4AllEmbeddings())

In [ ]:
question = "Where are the fires located?"
docs = vectorstore.similarity_search(question)
len(docs)

In [ ]:
# RAG prompt 
from langchain import hub
QA_CHAIN_PROMPT = hub.pull("rlm/rag-prompt-llama")

In [ ]:
# connect with the LLM, in this case llama2
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler                                  
llm = Ollama(model="llama2", 
             callback_manager = CallbackManager([StreamingStdOutCallbackHandler()]))

In [ ]:
# QA chain
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

In [ ]:
question = "How forest fires are active and what are they named?"
result = qa_chain({"query": question})